 # Analysis Justifaction

In [1]:
import pandas as pd

In [ ]:
import pandas as pd

# Read your datasets
fact_journey_metrics = pd.read_csv("MTA_Bus_Customer_Journey-Focused_Metrics.csv")
fact_fare_evasion = pd.read_csv("MTA_Bus_Fare_Evasion.csv")
fact_ridership_hourly = pd.read_csv("mta_bus_hourly_sampled.csv")
fact_service_delivered = pd.read_csv("MTA_Bus_Service_Delivered.csv")
fact_speeds = pd.read_csv("MTA_Bus_Speeds.csv")
fact_wait_assessment = pd.read_csv("MTA_Bus_Wait_Assessment.csv")
dim_stop = pd.read_csv("MTA_Bus_Timepoints_20250518.csv")  # used as lookup


# Convert transit_timestamp to datetime
fact_ridership_hourly['transit_timestamp'] = pd.to_datetime(fact_ridership_hourly['transit_timestamp'])

# Extract date and hour
fact_ridership_hourly['date'] = fact_ridership_hourly['transit_timestamp'].dt.date
fact_ridership_hourly['hour'] = fact_ridership_hourly['transit_timestamp'].dt.hour

# Now create dim_time
dim_time = fact_ridership_hourly[['date', 'hour']].drop_duplicates()
dim_time['date'] = pd.to_datetime(dim_time['date'])
dim_time['period'] = dim_time['hour'].apply(lambda h: 'AM' if h < 12 else 'PM')
dim_time['month'] = dim_time['date'].dt.month
dim_time['quarter'] = dim_time['date'].dt.quarter



In [9]:
fact_service_delivered = fact_service_delivered[[
    'route_id', 'borough', 'trip_type', 'period',
    'scheduled_number_of_buses', 'actual_number_of_buses', 'service_delivered'
]]


In [27]:
import pandas as pd
import os

# Define paths to your CSVs
csv_files = {
    "fact_journey_metrics": "MTA_Bus_Customer_Journey-Focused_Metrics.csv",
    "fact_fare_evasion": "MTA_Bus_Fare_Evasion.csv",
    "fact_ridership_hourly": "mta_bus_hourly_sampled.csv",
    "fact_service_delivered": "MTA_Bus_Service_Delivered.csv",
    "fact_speeds": "MTA_Bus_Speeds.csv",
    "fact_timepoints": "MTA_Bus_Timepoints_20250518.csv",
    "fact_wait_assessment": "MTA_Bus_Wait_Assessment.csv"
}

# Convert all to parquet
for name, path in csv_files.items():
    df = pd.read_csv(path)
    df.to_parquet(f"{name}.parquet", index=False)
    print(f"✅ Converted {path} → {name}.parquet")


✅ Converted MTA_Bus_Customer_Journey-Focused_Metrics.csv → fact_journey_metrics.parquet
✅ Converted MTA_Bus_Fare_Evasion.csv → fact_fare_evasion.parquet
✅ Converted mta_bus_hourly_sampled.csv → fact_ridership_hourly.parquet
✅ Converted MTA_Bus_Service_Delivered.csv → fact_service_delivered.parquet
✅ Converted MTA_Bus_Speeds.csv → fact_speeds.parquet
✅ Converted MTA_Bus_Timepoints_20250518.csv → fact_timepoints.parquet
✅ Converted MTA_Bus_Wait_Assessment.csv → fact_wait_assessment.parquet


In [42]:
# Safe renaming: only if the column exists
rename_map = {}
if 'Time Period' in fact_fare_evasion.columns:
    rename_map['Time Period'] = 'period'
if 'Trip Type' in fact_fare_evasion.columns:
    rename_map['Trip Type'] = 'trip_type'
if rename_map:
    fact_fare_evasion = fact_fare_evasion.rename(columns=rename_map)


In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm


# Optional memory reducer
def reduce_memory(df):
    for col in df.select_dtypes(include=['int', 'float']).columns:
        df[col] = pd.to_numeric(df[col], downcast='unsigned' if 'int' in str(df[col].dtype) else 'float')
    for col in df.select_dtypes(include=['object']).columns:
        num_unique = df[col].nunique()
        num_total = len(df[col])
        if num_unique / num_total < 0.5:
            df[col] = df[col].astype('category')
    return df

# -------------------------------
# 1. Load parquet files
# -------------------------------
fact_journey_metrics     = pd.read_parquet("fact_journey_metrics.parquet")
fact_wait_assessment     = pd.read_parquet("fact_wait_assessment.parquet")
fact_speeds              = pd.read_parquet("fact_speeds.parquet")
fact_service_delivered   = pd.read_parquet("fact_service_delivered.parquet")
fact_fare_evasion        = pd.read_parquet("fact_fare_evasion.parquet")

# Rename for consistency
fact_fare_evasion = fact_fare_evasion.rename(columns={
    'Time Period': 'period',
    'Fare Evasion': 'fare_evasion_rate'
})

# Reduce memory
fact_wait_assessment   = reduce_memory(fact_wait_assessment)
fact_speeds            = reduce_memory(fact_speeds)
fact_service_delivered = reduce_memory(fact_service_delivered)
fact_fare_evasion      = reduce_memory(fact_fare_evasion)

# -------------------------------
# 2. Create service lookup per (borough, route_id)
# -------------------------------
service_lookup = {
    (boro, route): df
    for (boro, route), df in fact_service_delivered.groupby(['borough', 'route_id'])
}


# -------------------------------
# 3. Merge in Chunks
# -------------------------------
final_chunks = []

group_cols = ['borough', 'route_id']
journey_groups = fact_journey_metrics.groupby(group_cols)

for (boro, route), df in tqdm(journey_groups, desc="Merging chunks"):
    df = df.copy()  # prevent SettingWithCopyWarning

    # Merge wait assessment
    df = df.merge(
        fact_wait_assessment[['route_id','borough','trip_type','period','wait_assessment']],
        on=['route_id','borough','trip_type','period'],
        how='left'
    )

    # Merge speeds
    df = df.merge(
        fact_speeds[['route_id','borough','trip_type','period','average_speed']],
        on=['route_id','borough','trip_type','period'],
        how='left'
    )

    # Merge service delivered (safe per-route)
    service_df = service_lookup.get((boro, route))
    if service_df is not None and not service_df.empty:
        df = df.merge(
            service_df[[
                'route_id', 'borough', 'trip_type', 'period',
                'scheduled_number_of_buses', 'actual_number_of_buses', 'service_delivered'
            ]],
            on=['route_id','borough','trip_type','period'],
            how='left'
        )

    # Merge fare evasion
    fact_fare_evasion = fact_fare_evasion.rename(columns={
    'Time Period': 'period',
    'Trip Type': 'trip_type',
    'Fare Evasion': 'fare_evasion_rate'
    })


    # Reduce memory per chunk
    df = reduce_memory(df)

    final_chunks.append(df)

# -------------------------------
# 4. Finalize
# -------------------------------
merged_df = pd.concat(final_chunks, ignore_index=True)
merged_df = reduce_memory(merged_df)

# Save as compressed Parquet
merged_df.to_parquet("merged_mta_bus_data_final.parquet", index=False, compression='snappy')


C:\Users\DELL\AppData\Local\Temp\ipykernel_15080\735602367.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (boro, route), df in fact_service_delivered.groupby(['borough', 'route_id'])
Merging chunks: 100%|██████████| 343/343 [00:46<00:00,  7.36it/s]


In [47]:
merged_df.shape

(45767880, 14)

In [48]:
merged_df

,month,borough,trip_type,route_id,period,number_of_customers,additional_bus_stop_time,additional_travel_time,customer_journey_time_performance,wait_assessment,average_speed,scheduled_number_of_buses,actual_number_of_buses,service_delivered
0,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,2.392148e+05,1.307586,0.112321,0.787305,0.843048,7.01,NaN,NaN,NaN
1,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,2.392148e+05,1.307586,0.112321,0.787305,0.843048,7.02,NaN,NaN,NaN
2,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,2.392148e+05,1.307586,0.112321,0.787305,0.843048,6.89,NaN,NaN,NaN
3,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,2.392148e+05,1.307586,0.112321,0.787305,0.843048,6.80,NaN,NaN,NaN
4,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,2.392148e+05,1.307586,0.112321,0.787305,0.843048,6.92,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45767875,12/1/2024,Systemwide,Systemwide,Systemwide,Systemwide,4.432384e+07,2.153359,0.668039,0.699376,0.754794,8.57,48713.0,46962.0,0.964055
45767876,12/1/2024,Systemwide,Systemwide,Systemwide,Systemwide,4.432384e+07,2.153359,0.668039,0.699376,0.754794,8.57,202843.0,193391.0,0.953402
45767877,12/1/2024,Systemwide,Systemwide,Systemwide,Systemwide,4.432384e+07,2.153359,0.668039,0.699376,0.754794,8.57,59831.0,58262.0,0.973776
45767878,12/1/2024,Systemwide,Systemwide,Systemwide,Systemwide,4.432384e+07,2.153359,0.668039,0.699376,0.754794,8.57,221218.0,210288.0,0.950592


In [2]:
merged_df = pd.read_parquet('merged_mta_bus_data_final.parquet')

In [3]:
import dask.dataframe as dd
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
import os

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno

def missing_summary(df, df_name='DataFrame', plot=False):
    """
    Print a summary of missing values in a DataFrame and optionally plot missingness.

    Parameters:
    - df: pandas DataFrame
    - df_name: str, name of the DataFrame for labeling
    - plot: bool, whether to show missing data visualizations
    """
    print(f"\n{'='*80}")
    print(f"🕳️ Missing Value Summary for: {df_name}")

    # Count missing values
    missing = df.isna().sum()
    missing = missing[missing > 0]

    if missing.empty:
        print("✅ No missing values.")
        return

    # Calculate and display missing percentages
    missing_percent = (missing / len(df)) * 100
    summary = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_percent.round(2)
    }).sort_values(by='Missing %', ascending=False)

    print(summary.to_string())

    # Show visualizations
    if plot:
        plt.figure(figsize=(12, 4))
        msno.matrix(df)
        plt.title(f'Missing Value Matrix: {df_name}')
        plt.show()

        plt.figure(figsize=(6, 4))
        msno.heatmap(df)
        plt.title(f'Missing Value Heatmap: {df_name}')
        plt.show()


In [4]:
merged_df.dtypes

month                                category
borough                              category
trip_type                            category
route_id                             category
period                               category
number_of_customers                   float64
additional_bus_stop_time              float32
additional_travel_time                float32
customer_journey_time_performance     float32
wait_assessment                       float32
average_speed                         float32
scheduled_number_of_buses             float32
actual_number_of_buses                float32
service_delivered                     float32
dtype: object

In [10]:
# Copy to avoid modifying in place (optional safety)
df = merged_df.copy()

# --------------------------------------
# 1. Flag missing service delivery info
# --------------------------------------
df['missing_service_info'] = df['scheduled_number_of_buses'].isna().astype('uint8')

# --------------------------------------
# 2. Fill missing: Service Delivered Metrics
# Group-based median imputation
# --------------------------------------
service_cols = ['scheduled_number_of_buses', 'actual_number_of_buses', 'service_delivered']
for col in service_cols:
    df[col] = df.groupby(['route_id', 'borough', 'trip_type'])[col]\
                .transform(lambda x: x.fillna(x.median()))
    
    # If any value still missing, fill with overall median
    df[col] = df[col].fillna(df[col].median())

# --------------------------------------
# 3. Fill missing: Average Speed (use group mean)
# --------------------------------------
df['average_speed'] = df.groupby(['route_id', 'borough', 'trip_type'])['average_speed']\
                        .transform(lambda x: x.fillna(x.mean()))
df['average_speed'] = df['average_speed'].fillna(df['average_speed'].mean())

# --------------------------------------
# 4. Fill missing: Wait Assessment (use group mean)
# --------------------------------------
df['wait_assessment'] = df.groupby(['route_id', 'borough', 'trip_type'])['wait_assessment']\
                           .transform(lambda x: x.fillna(x.mean()))
df['wait_assessment'] = df['wait_assessment'].fillna(df['wait_assessment'].mean())

# --------------------------------------
# 5. Final check: No missing values should remain
# --------------------------------------
assert df.isna().sum().sum() == 0, "⚠️ Some missing values remain!"

# Optional: Save cleaned version
df.to_parquet("merged_mta_bus_data_cleaned.parquet", index=False, compression='snappy')


C:\Users\DELL\AppData\Local\Temp\ipykernel_8712\4023315227.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[col] = df.groupby(['route_id', 'borough', 'trip_type'])[col]\
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\Local

In [5]:
df = pd.read_parquet('merged_mta_bus_data_cleaned.parquet')

In [7]:
# Load dim_stop
dim_stop = pd.read_parquet("fact_timepoints.parquet")

# Drop rows with missing Lat/Long
dim_stop = dim_stop.dropna(subset=['Latitude', 'Longitude'])

# Group by Route ID: get average lat/lon and first georeference
route_geo = (
    dim_stop.groupby('Route ID')
    .agg({
        'Latitude': 'mean',
        'Longitude': 'mean',
        'Georeference': 'first'  # Or use mode if needed
    })
    .reset_index()
)

# Rename to match merged_df
route_geo = route_geo.rename(columns={'Route ID': 'route_id'})

# Merge into merged_df
BUS = df.merge(route_geo, on='route_id', how='left')


In [8]:
bus = BUS.dropna(subset=['Latitude', 'Longitude', 'Georeference'])


In [7]:
bus.to_parquet("bus_MTA_Merged_Georefranced.parquet", index=False, compression='snappy')


In [9]:
bus.shape

(45601980, 18)

In [10]:
bus.isna().sum()

month                                0
borough                              0
trip_type                            0
route_id                             0
period                               0
number_of_customers                  0
additional_bus_stop_time             0
additional_travel_time               0
customer_journey_time_performance    0
wait_assessment                      0
average_speed                        0
scheduled_number_of_buses            0
actual_number_of_buses               0
service_delivered                    0
missing_service_info                 0
Latitude                             0
Longitude                            0
Georeference                         0
dtype: int64

In [12]:
bus

,month,borough,trip_type,route_id,period,number_of_customers,additional_bus_stop_time,additional_travel_time,customer_journey_time_performance,wait_assessment,average_speed,scheduled_number_of_buses,actual_number_of_buses,service_delivered,missing_service_info,Latitude,Longitude,Georeference
0,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,239214.785000,1.307586,0.112321,0.787305,0.843048,7.01,547.0,512.5,0.972252,1,40.855875,-73.904067,POINT (-73.896434 40.862958)
1,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,239214.785000,1.307586,0.112321,0.787305,0.843048,7.02,547.0,512.5,0.972252,1,40.855875,-73.904067,POINT (-73.896434 40.862958)
2,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,239214.785000,1.307586,0.112321,0.787305,0.843048,6.89,547.0,512.5,0.972252,1,40.855875,-73.904067,POINT (-73.896434 40.862958)
3,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,239214.785000,1.307586,0.112321,0.787305,0.843048,6.80,547.0,512.5,0.972252,1,40.855875,-73.904067,POINT (-73.896434 40.862958)
4,1/1/2024,Bronx,LCL/LTD,BX1,Off-Peak,239214.785000,1.307586,0.112321,0.787305,0.843048,6.92,547.0,512.5,0.972252,1,40.855875,-73.904067,POINT (-73.896434 40.862958)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45601987,12/1/2024,Staten Island,EXP,SIM9,Peak,3833.312988,3.415846,-1.352230,0.635087,0.609884,15.90,396.0,371.0,0.936869,0,40.621493,-74.077360,POINT (-74.165154 40.574887)
45601988,12/1/2024,Staten Island,EXP,SIM9,Peak,3833.312988,3.415846,-1.352230,0.635087,0.609884,15.90,18.0,16.0,0.888889,0,40.621493,-74.077360,POINT (-74.165154 40.574887)
45601989,12/1/2024,Staten Island,EXP,SIM9,Peak,3833.312988,3.415846,-1.352230,0.635087,0.609884,15.90,339.0,330.0,0.973451,0,40.621493,-74.077360,POINT (-74.165154 40.574887)
45601990,12/1/2024,Staten Island,EXP,SIM9,Peak,3833.312988,3.415846,-1.352230,0.635087,0.609884,15.90,18.0,19.0,1.055556,0,40.621493,-74.077360,POINT (-74.165154 40.574887)


 # Numrical & Categorical Columns Inspection

In [18]:
def summarize_columns(df):
    print(f"{'='*80}")
    print("📊 Numerical Columns - Min & Max:")
    print(f"{'='*80}")
    
    numeric_cols = df.select_dtypes(include=['float32', 'float64', 'int']).columns
    for col in numeric_cols:
        min_val = df[col].min()
        max_val = df[col].max()
        print(f"{col:35} ➤ Min: {min_val:10.2f} | Max: {max_val:10.2f}")
    
    print(f"\n{'='*80}")
    print("🔢 Categorical Columns - Unique Value Counts:")
    print(f"{'='*80}")
    
    cat_cols = df.select_dtypes(include='category').columns
    for col in cat_cols:
        unique_vals = df[col].nunique(dropna=True)
        print(f"{col:35} ➤ Unique Values: {unique_vals}")

# Example usage
summarize_columns(bus)


📊 Numerical Columns - Min & Max:
number_of_customers                 ➤ Min:       1.23 | Max:  458435.07
additional_bus_stop_time            ➤ Min:       0.11 | Max:      18.06
additional_travel_time              ➤ Min:     -12.59 | Max:      16.84
customer_journey_time_performance   ➤ Min:       0.00 | Max:       0.95
wait_assessment                     ➤ Min:       0.27 | Max:       1.00
average_speed                       ➤ Min:       1.40 | Max:      33.77
scheduled_number_of_buses           ➤ Min:       5.00 | Max:    2890.00
actual_number_of_buses              ➤ Min:       0.00 | Max:    2751.00
service_delivered                   ➤ Min:       0.00 | Max:       1.35
Latitude                            ➤ Min:      40.55 | Max:      40.89
Longitude                           ➤ Min:     -74.19 | Max:     -73.74

🔢 Categorical Columns - Unique Value Counts:
month                               ➤ Unique Values: 12
borough                             ➤ Unique Values: 5
trip_type         

 - Number_of_customers
  Conclusion
    - These are clearly monthly totals, but specifically for the entire NYC Bus System.

    - The extremely high values (e.g., 44 million customers) are correct for systemwide monthly ridership. For context:

    - 🚍 MTA Bus ridership for the entire NYC system is about 1.2–1.4 million passengers per weekday → ~40 million per month is reasonable for total counts.

    - So your max value is valid — it’s just from the “Systemwide” rows

        - Action 
            -  What is "Systemwide"?
                    -  "Systemwide" is a placeholder value used in categorical columns like borough, route_id, or trip_type to represent aggregate metrics across the entire MTA bus network — rather than data tied to a specific borough, route, or time period.

                    - It summarizes total or average performance for all routes combined.

                    - Appears in rows where all location identifiers are “Systemwide”.

                    - Often used in dashboards or reports for executive-level overview.
            - Dropping “Systemwide” rows improves analytical clarity by focusing on route- and borough-specific metrics — which are more actionable for performance, planning, or geographic studies.

In [19]:
# Drop all rows where any of these columns contain 'Systemwide'
bus = bus[
    (bus['borough'] != 'Systemwide') &
    (bus['route_id'] != 'Systemwide') &
    (bus['trip_type'] != 'Systemwide')
]

# Optionally save the cleaned dataset
# bus.to_parquet("bus_MTA_Merged_Georefranced_cleaned.parquet", index=False, compression='snappy')


 - scheduled_number_of_buses
    - 🧠 Interpretation:

        - This may not mean “2,890 physical buses”, but “2,890 bus runs” (i.e., scheduled trips).

        - In transit datasets, scheduled_number_of_buses often means total runs/trips per month per route, not vehicles.

        - If a bus route runs 100 times per weekday → 100 × 20 workdays = 2,000 trips per month is plausible.

    - 🟢 So, is 2,890 reasonable?

        - Yes, if the number refers to total bus trips (e.g., Q46 has many daily trips). For reference:

        - MTA runs frequent service on busy routes like Q46

        - If a bus runs every 10 minutes during peak over 6 hours per weekday: 6 × 6 = 36 trips/day × 22 weekdays = 792 trips

        - Add weekends and off-peak → reaching 2,000–3,000 trips/month is possible

    - ✅ Conclusion:

        - The value is not erroneous.

        - It reflects total scheduled trips — not number of physical buses.

        - No action needed. But you can rename the column for clarity.